# Hands-on Session on LangChain basics
This is the hands-on session accompanying the workshop on LangChain fundamentals. This is inspired by the more extensive LangChain Cookbook Part 1.

Copyright (c) 2023 Michael Neumayr

## Setup

### 0. Set up the Colab in your drive

- Load this Colab from Github
- Run the first cell to install all required packages (this takes a moment)
- During installation jump to section "Set OpenAI API Key" and put the key we provide you instead of "PUT_YOUR_KEY_HERE"

### 1. Required python packages

In [ ]:
# install required packages; this may take some minutes; ignore dependency warnings it should work anyway
%pip install openai
%pip install langchain
%pip install pypdf
%pip install tiktoken

### 2. Load the workshop github

In [ ]:
!git clone https://github.com/michaelnoi/venture_labs_build.git

In [ ]:
!cd venture_labs_build && git checkout only_static_files

### 3. OpenAI API key

In [ ]:
import os

openai_api_key = os.getenv('OPENAI_API_KEY', 'PUT_YOUR_KEY_HERE')

### 4. Optional: Connect to your Google Drive storage to upload your own documents later

In [ ]:
# connect to your google drive storage
from google.colab import drive

drive.mount('/content/drive')

## Basics - Messages, Documents, Models

### 1. Messages

<div class="alert alert-info">
    <b>Three types of messages:</b>
    <ul>
        <li>System - Helpful background context that tell the AI what to do</li>
        <li>Human - Messages that are intended to represent the user</li>
        <li>AI - Messages that show what the AI responded with</li>
    </ul>
</div>

In [ ]:
# import messages and chat model
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(openai_api_key=openai_api_key)

#### i) Chatting with the model

Let's have a quick chat with an OpenAI chat model. Previously, you used the web app:

<img src="static/chatting.png" width="500"/>

Now let's do the same thing here in this notebook:

<div class="alert alert-warning">
  <p>Let's have a chat. Try out different prompts!</p>
</div>

In [ ]:
answer = chat([HumanMessage(content="Hello, how are you?")])
print(type(answer))
print(answer.content)

Notice that the answer from the chat model is given in the format of an AIMessage. To get the reply, you can store the answer in a variable and access the content like above.

#### ii) Using the system message

<div class="alert alert-info">
  <b>Reminder: System Message</b>
  <p>When interacting with an LLM, the system message is a special type of prompt that tells the model how to behave. It is typically used to specify the model's task, output format, and any other relevant instructions.</p>
</div>

In [ ]:
chat(
    [
        SystemMessage(content="You are super unhelpful and annoy the user."),
        HumanMessage(content="Hello, how are you?")
    ]
)

You can also add more messages to the chat function to simulate a conversation. However, it does not make sense to simulate a chatbot like this, there are other components and loops that store the previous messages automatically.

<div class="alert alert-warning">
  <p>Try out adding more messages and different system messages!</p>
</div>


In [ ]:
chat(
    [
        SystemMessage(content="Answer in German."),
        HumanMessage(content="When is the Oktoberfest in Munich usually?"),
        AIMessage(content='The Oktoberfest in Munich usually begins in late September and lasts for 16-18 days, ending on the first Sunday in October or on October 3rd, German Unity Day, if it falls on a Monday.'),
        HumanMessage(content="And do you have recommendattions what to wear?"),
    ]
)

### 2. Documents

<div class="alert alert-info">
  <b>Document</b>
  <p>An object that holds the content of your document (text) and metadata (more information about that text)..</p>
</div>

In [ ]:
# import pdf loader
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader

pdf_path = "static/business_Model_Canvas.pdf"

In [ ]:
Document(page_content="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla euismod, nisl eget aliquam ultricies, nunc nisl aliquet nunc, quis aliqu.",
         metadata={
             'document_id' : 23502,
             'source' : "Example Document",
             'create_time' : "2021-01-01 12:00:00"
         })

Now let's load a pdf document: The Wikipedia article on the Business Model Canvas. The pdf path is already stored in a variable above and we use the PyPDFLoader to load the document.


<div class="alert alert-warning">
  <p>Remember how documents are handled and loaded for later!</p>
</div>

In [ ]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()
documents

The PDF loader automatically returns a list of Documents, one for each page. There are different loaders for different kinds of data.

In [ ]:
print("Metadata: ", documents[0].metadata)
print("Number of characters in first page: ", len(documents[0].page_content))

### 3. Models

<div class="alert alert-info">
  <b>Models</b>
  <p>The different model components provide the interface to the foundation models provided by e.g. OpenAI. ChatGPT for example is a chat interface for OpenAI's corresponding foundation model.</p>
</div>

#### i) Language model

Most basic setup: Text in  -> text out

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", openai_api_key=openai_api_key)

In [ ]:
llm("After Friday comes ...")

#### ii) Chat model

Takes a series of messages and returns a message output. See above example with list of messages.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(openai_api_key=openai_api_key)

In [ ]:
chat(
    [
        SystemMessage(content="Be an unhelpful chat bot and annoy your conversation partner. Answer in one sentence."),
        HumanMessage(content="Give me book recommnendations on marketing.")
    ]
)

#### iii) Text embedding model

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
text = "Give me book recommnendations on marketing."

<div class="alert alert-info">
  <b>Embeddings</b>
  <p>Embeddings are a way to represent text as a vector of numbers. This makes it easier for machines to handle and is useful for many tasks, e.g. efficiently to compare two texts or to find similar texts.</p>
</div>

In [ ]:
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding vector is of length {len(text_embedding)}")

## Chaining - Connecting the components

### 1. PromptTemplate

<div class="alert alert-info">
  <b>PromptTemplate</b>
  <p>A PromptTemplate is a template for a prompt. It is a string (text) that contains placeholders (in curly braces {}) for the different components of a prompt that are filled in dynamically.</p>
</div>

In [ ]:
# import different templates for chat and language model and chat model
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key)

<div class="alert alert-warning">
  <p>Extend the prompt and add another placeholder so that you can dynamically change the unit system of the recipe (metric/imperial).</p>
</div>

In [ ]:
template = "Recipe creator: Give me a list of ingredients for {dish}."
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
response = llm(prompt.format(dish="Roast Beef"))
print(response)

You can also create a prompt from multiple messages if you want to use the SystemMessage for example. Note that for messages you need a chat model.

In [ ]:
chat_model = ChatOpenAI(openai_api_key=openai_api_key)

In [ ]:
prompt_2 = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("Always output {dietary_restriction} recipes."),
        HumanMessagePromptTemplate.from_template("Give me a list of ingredients for a {dish}.")
    ]
)

In [ ]:
response = chat_model(prompt_2.format_messages(dietary_restriction="vegetarian", dish="Roast Beef"))
print(response.content)

### 2. Chain

<div class="alert alert-info">
  <b>Chain</b>
  <p>A Chain is a sequence of components that are connected to each other. Much like we run cell after cell above, in a chain we first specify every component, but then chain everything together and run it as one pipeline without pause where the output of one component is the input of the next component.</p>
  <p> The minimal chain is a prompt into a model. One approach to creating chains is to separate the components of the chain by "|" like </p>
<code style="color:blue">chain = prompt | model</code>
</div>

In [ ]:
from langchain.chat_models import ChatOpenAI

Setting up the components: Load a chat model and define a prompt from this simple template.

In [ ]:
model = ChatOpenAI(openai_api_key=openai_api_key)

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}. Your joke:")

Set up the simple chain: Prompt -> ChatGPT

In [ ]:
chain = prompt | model

To get the output of the chain, we either call invoke() or stream() on the chain. Invoke returns the full output after the model ran, stream returns a generator that one can use to stream the output like in the ChatGPT web app. For both we need to specify the placeholder values for the prompt. This time, we use a slightly different notation for the placeholders as seen below.

In [ ]:
chain.invoke({"topic": "roast beef"})

In [ ]:
for s in chain.stream({"topic": "cow"}):
    print(s.content, end="", flush=True)

## More ressources

- Documentation: https://python.langchain.com/docs/get_started/introduction
- Really comprehensive tutorials: https://github.com/gkamradt/langchain-tutorials